# Fidelity Estimation using MS Unitary

## MS Unitary

MS Hamiltonian:
    
$$
\begin{aligned}
H_{MS} & = \sum_i^N \Bigg( \Omega_i \cos\{\omega_dt-\phi_m\}\sigma^{(\phi_s)}
\\
& + \frac{\Omega_i}{2} \sigma^{(\pi/2+\phi_s)} \sum_k \eta^k_i \left[ e^{-i\omega_dt+i\phi_m+i\nu_k t} a^\dagger_k + \text{h.c.} \right]
\\
& + \frac{\Omega_i}{2} \sigma^{(\pi/2+\phi_s)} \sum_k \eta^k_i \left[ e^{i\omega_dt-i\phi_m+i\nu_k t} a^\dagger_k + \text{h.c.} \right] \Bigg)
\end{aligned} 
$$

We only consider the second term in parentheses (the second line). The first term is direct coupling to qubit transition, which should be eliminable through RWA so long as $\omega_d$ is large enough. The last term is off-resonant coupling from red sideband lasers to blue sidebands and vice versa, which should have an even smaller effect than direct coupling.

The unitary for the second term is given by the magnus expansion:

$$
\begin{align}
    M_1(t) & = \sum_{i,k} \frac{\Omega_i \eta^k_i}{2({\omega_d-\nu_k})} \sigma^{(\pi/2+\phi_s)}_i
    \left( (e^{-i(\omega_d-\nu_k)t}-1)e^{i\phi_m}a^\dagger_k - \text{h.c.} \right)
\end{align}
$$

$$
\begin{align}
    M_2(t) & = -\frac{i}{4} \sum_{k,i_1, i_2} \Omega_{i_1}\Omega_{i_2}\eta^k_{i_1}\eta^k_{i_2} \sigma^{(\pi/2+\phi_s)}_{i_1}\sigma^{(\pi/2+\phi_s)}_{i_2}
    \left(\frac{t}{\omega_d-\nu_k}-\frac{\sin\{(\omega_d-\nu_k)t\}}{(\omega_d-\nu_k)^2} \right)
\end{align}
$$

$$
\begin{align}
    U(t) = \exp\left(M_1(t)+M_2(t)\right)
\end{align}
$$

## QuTiP Implementation

In [1]:
import qutip as qtp
from qutip import sigmax, sigmay, sigmaz, tensor, qeye, create, destroy, displace, basis, fock, ket2dm

import numpy as np
from numpy import cos, sin, exp, pi as π, sqrt

from math import factorial

import itertools

import tools.IonChainTools as ict
import tools.MSFidelityEstimation as msfe

In [2]:
ψbell1 = (tensor(basis(2,0),basis(2,0))-1j*tensor(basis(2,1),basis(2,1)))/sqrt(2)
ψbell2 = (tensor(basis(2,0),basis(2,0))+1j*tensor(basis(2,1),basis(2,1)))/sqrt(2)
ψbell3 = (tensor(basis(2,0),basis(2,0))+tensor(basis(2,1),basis(2,1)))/sqrt(2)

In [3]:
def embedop(O, N, i, d=2):
    '''
    Embed operator O on subspace of dimension d in space of dimension N^d with i as index of subsystem
    '''
    return tensor([O if j==i else qeye(d) for j in range(N)])

In [4]:
def σ(ϕ):
    return cos(ϕ)*sigmax()+sin(ϕ)*sigmay()

In [5]:
def gen_intensity_vector_fixedxt(N, targets, neighbor_intensity, next_neighbor_intensity):
    efields = np.zeros(N)
    for t in targets:
        efields[t] +=1
        for n in (t-1, t+1):
            if n>=0 and n<N:
                efields[n] += np.sqrt(neighbor_intensity)
        for nn in (t-2, t+2):
            if nn>=0 and nn<N:
                efields[nn] += np.sqrt(next_neighbor_intensity)
    intensities = efields**2
    return intensities

In [6]:
def UMS(t, N, Ωvals, η, ωd, modenums=None, modetype="radial", νz=1e6, νr=3e6, ϕs=π/2, ϕm=0, modetrunc=5):
    # The MS unitary as a qutip operator, acting on both the qubit and motional mode spaces
    if modetype == "radial":
        modes = ict.calcRadialModes(N, νratio=νr/νz)
    elif modetype == "axial":
        modes = ict.calcAxialModes(N)
    else:
        raise Exception("Only modetype options are 'radial' and 'axial'")
    
    if modenums == None:
        modenums = list(range(N))
                
    def M1_summation_term(k, i):
        ηik = η*modes[k][1][i]
        νk = modes[k][0]*νz
        strength = ηik*Ωvals[i]/(2*(ωd-νk))
        σi = embedop(σ(π/2+ϕs), N, i)
        α = (exp(-1j*(ωd-νk)*t)-1)*exp(1j*ϕm)
        ak = embedop(destroy(modetrunc), len(modenums), modenums.index(k), modetrunc)
        #print('α', α)
        return tensor(strength*σi, (α*ak - α.conjugate()*ak.dag()))
        # Dα = embedop(displace(modetrunc, α), len(modenums), modenums.index(k), modetrunc)
        # return strength*tensor(σi,Dα)
    
    def M2_summation_term(k, i1, i2):
        ηki1 = η*modes[k][1][i1]
        ηki2 = η*modes[k][1][i2]
        νk = modes[k][0]*νz
        strength = Ωvals[i1]*Ωvals[i2]*ηki1*ηki2
        σi1 = embedop(σ(π/2+ϕs), N, i1)
        σi2 = embedop(σ(π/2+ϕs), N, i2)
        time_dependence = t/(ωd-νk) - sin((ωd-νk)*t)/(ωd-νk)**2
        modeeye = tensor([qeye(modetrunc) for _ in modenums])
        #targetstrength = η**2/3*Ωvals[0]*Ωvals[1]
        #print(i1, i2, strength/targetstrength)
        #print(1/4*strength*time_dependence)
        return -1j/4 * strength * tensor(σi1*σi2, modeeye) * time_dependence

    completeM1 = 0
    for k in modenums:
        for i in range(N):
            completeM1 += M1_summation_term(k, i)
            
    completeM2 = 0
    for k in modenums:
        for i1 in range(N):
            for i2 in range(N):
                completeM2 += M2_summation_term(k, i1, i2)
            
    U = (completeM1+completeM2).expm()

    return U

In [7]:
def MSStateFromUnitary(N, m, targets, Ωvals, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L=1):
    # Act the MS unitary on an initial state of |0> qubits and no motional excitation
    if modetype == "radial":
        modes = ict.calcRadialModes(N, νratio=νr/νz)
    elif modetype == "axial":
        modes = ict.calcAxialModes(N)
    else:
        raise Exception("Only modetype options are 'radial' and 'axial'")

    ηmi1 = η*modes[m][1][i1]
    ηmi2 = η*modes[m][1][i2]

    # δ = sqrt(abs(2*K*Ωvals[i1]*Ωvals[i2]*ηmi1*ηmi2))
    # print("δ : ", δ)
    # τ = 2*π*K/δ
    # νm = modes[m][0]*νz
    # ωd = νm + δ

    δ = sqrt(abs(4*K*Ωvals[i1]*Ωvals[i2]*ηmi1*ηmi2))
    τ = 2*π*K/δ
    νm = modes[m][0]*νz
    ωd = νm + δ

    # strength = abs(Ωvals[i1]*Ωvals[i2]*ηmi1*ηmi2)
    # time_dependence = τ/(ωd-νm) - sin((ωd-νm)*τ)/(ωd-νm)**2
    # θ = -1/4 * strength * time_dependence

    t=τ*L

    ψ0 = tensor(tensor([basis(2,0) for _ in range(N)]), tensor([fock(modetrunc,0) for _ in modenums]))
    U = UMS(t, N, Ωvals, η, ωd, modenums=modenums, modetype=modetype, νz=νz, νr=νr, ϕs=ϕs, ϕm=ϕm, modetrunc=modetrunc)
    ψf = U * ψ0
    
    return ψf

In [8]:
def MSPerfectCompleteStateFidelity_FromUnitary(N, m, targets, intensities, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L=1):
    # Use numerically calculated MS unitary to compare final state with ideal final state, including all qubits and modes
    if modetype == "radial":
        modes = ict.calcRadialModes(N, νratio=νr/νz)
    elif modetype == "axial":
        modes = ict.calcAxialModes(N)
    else:
        raise Exception("Only modetype options are 'radial' and 'axial'")
    targetgatedirection = np.sign(modes[m][1][targets[0]]*modes[m][1][targets[1]])
    ψf = MSStateFromUnitary(N, m, targets, intensities, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L)
    idealUMS = tensor((-1j*targetgatedirection*π/4*tensor([sigmax() if i in (i1, i2) else qeye(2) for i in range(N)])).expm(), *[qeye(modetrunc) for _ in modenums])
    ψ0 = tensor(tensor([basis(2,0) for _ in range(N)]), tensor([fock(modetrunc,0) for _ in modenums]))
    idealψf = idealUMS*ψ0
    return abs((ψf.dag()*idealψf).data[0,0])**2

In [1]:
def MSPerfectQubitStateFidelity_FromUnitary(N, m, targets, intensities, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L=1):
    # Use numerically calculated MS unitary to compare final qubit state with ideal final qubit state, tracing out modes
    if modetype == "radial":
        modes = ict.calcRadialModes(N, νratio=νr/νz)
    elif modetype == "axial":
        modes = ict.calcAxialModes(N)
    else:
        raise Exception("Only modetype options are 'radial' and 'axial'")
    targetgatedirection = np.sign(modes[m][1][targets[0]]*modes[m][1][targets[1]])
    ψf = MSStateFromUnitary(N, m, targets, intensities, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L)
    ρfqbit = ket2dm(ψf).ptrace(list(range(N))) # Trace out modes by tracing out everything except the N qubits
    idealUMS = (-1j*targetgatedirection*π/4*tensor([sigmax() if i in (i1, i2) else qeye(2) for i in range(N)])).expm()
    ψ0 = tensor([basis(2,0) for _ in range(N)])
    idealψfqbit = idealUMS*ψ0
    return (ρfqbit*ket2dm(idealψfqbit)).tr()

In [10]:
def MSTracedTargetsFidelity_FromUnitary(N, m, targets, intensities, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L=1):
    # Use numerically calculated MS unitary to compare final qubit state with ideal final qubit state, tracing out modes
    if modetype == "radial":
        modes = ict.calcRadialModes(N, νratio=νr/νz)
    elif modetype == "axial":
        modes = ict.calcAxialModes(N)
    else:
        raise Exception("Only modetype options are 'radial' and 'axial'")
    targetgatedirection = np.sign(modes[m][1][targets[0]]*modes[m][1][targets[1]])
    ψf = MSStateFromUnitary(N, m, targets, intensities, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L)
    targetsys = ψf.ptrace([i1,i2])
    idealUMS = (-1j*targetgatedirection*π/4*tensor(sigmax(), sigmax())).expm()
    ψ0 = tensor(basis(2,0), basis(2,0))
    idealψfqbit = idealUMS*ψ0
    return (targetsys*ket2dm(idealψfqbit)).tr()

## Analytical MS Error Calculation

In [11]:
def coherent_coeff(α, n):
    return exp(-abs(α)**2/2)*α**n/sqrt(factorial(n))

In [12]:
def MSPerfectStateFidelity_Analytical(N, m, targets, Ωvals, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L=1, fidtargets=None):
    # Analytically calculate final state and find fidelity with ideal final state, including both qubits and modes
    if modetype == "radial":
        modes = ict.calcRadialModes(N, νratio=νr/νz)
    elif modetype == "axial":
        modes = ict.calcAxialModes(N)
    else:
        raise Exception("Only modetype options are 'radial' and 'axial'")
    
    i1, i2 = targets
    
    ηmi1 = η*modes[m][1][i1]
    ηmi2 = η*modes[m][1][i2]

    δ = sqrt(abs(4*K*Ωvals[i1]*Ωvals[i2]*ηmi1*ηmi2))
    τ = 2*π*K/δ
    νm = modes[m][0]*νz
    ωd = νm + δ
    
    if fidtargets == None: fidtargets = targets
    fidtargetgatedirection = np.sign(modes[m][1][fidtargets[0]]*modes[m][1][fidtargets[1]])
    
    naturalbasis = list(itertools.product(*[[-1,+1]]*N))
    innerprod = 0
    t = τ*L
    for λ in naturalbasis:
        displacementproduct = 1
        for k in modenums:
            αk = np.sum([(Ωvals[i]*η*modes[k][1][i])/(2*(ωd-modes[k][0]*νz))*(exp(-1j*(ωd-modes[k][0]*νz)*t)-1)*exp(1j*ϕm)*λ[i] for i in range(N)])
            displacementproduct *= coherent_coeff(αk, 0)
            #print(λ, k, αk)
        phaseangle = 0
        for k in modenums:
            for j1, j2 in itertools.product(range(N),range(N)):
                ηkj1 = η*modes[k][1][j1]
                ηkj2 = η*modes[k][1][j2]
                νk = modes[k][0]*νz
                phaseangle += Ωvals[j1]*Ωvals[j2]*ηkj1*ηkj2*(t/(ωd-νk)-sin((ωd-νk)*t)/(ωd-νk)**2)*λ[j1]*λ[j2]
        M2phase = exp(-1j/4*phaseangle) # Should that be /2 or /4? Check.
        #print(λ, M2phase)
        correctphase = exp(1j*π/4*λ[fidtargets[0]]*λ[fidtargets[1]]*fidtargetgatedirection)
        innerprod += M2phase*correctphase*displacementproduct/2**(N)
    fidelity = abs(innerprod)**2
    return fidelity

In [15]:
def MSTracedTargetsFidelity_Analytical(N, m, targets, Ωvals, η, modetype, νz, νr, ϕs, ϕm, modenums, modetrunc, K, L=1, fidtargets=None):
    # Analytically calculate final qubit state and find fidelity with ideal final qubit state, with modes traced out
    if modetype == "radial":
        modes = ict.calcRadialModes(N, νratio=νr/νz)
    elif modetype == "axial":
        modes = ict.calcAxialModes(N)
    else:
        raise Exception("Only modetype options are 'radial' and 'axial'")
    
    i1, i2 = targets
    
    ηmi1 = η*modes[m][1][i1]
    ηmi2 = η*modes[m][1][i2]

    δ = sqrt(abs(4*K*Ωvals[i1]*Ωvals[i2]*ηmi1*ηmi2))
    τ = 2*π*K/δ
    νm = modes[m][0]*νz
    ωd = νm + δ
    
    ρqbit_mat = np.zeros((2**N,2**N), dtype=np.cdouble)
    
    naturalbasis = list(itertools.product(*[[-1,+1]]*N))
    innerprod = 0
    t = τ*L
    for λ1,λ2 in itertools.product(naturalbasis, naturalbasis): # λ is a N-element list, representing a qubit state in the X basis (-1=|0>, +1=|1>)
        λ1λ2coeff = 0
        for γ in list(itertools.product(*[list(range(modetrunc))]*len(modenums))): # γ is a list of N integers, representing a mode state in the number basis
            displacementproduct = 1/2**(N)
            for k in modenums:
                αλ1k = np.sum([(Ωvals[i]*η*modes[k][1][i])/(2*(ωd-modes[k][0]*νz))*(exp(-1j*(ωd-modes[k][0]*νz)*t)-1)*exp(1j*ϕm)*λ1[i] for i in range(N)])
                αλ2k = np.sum([(Ωvals[i]*η*modes[k][1][i])/(2*(ωd-modes[k][0]*νz))*(exp(-1j*(ωd-modes[k][0]*νz)*t)-1)*exp(1j*ϕm)*λ2[i] for i in range(N)])
                displacementproduct *= coherent_coeff(αλ1k, γ[modenums.index(k)])*coherent_coeff(αλ2k, γ[modenums.index(k)]).conjugate()
                #print(λ, k, αk)
            λ1λ2coeff += displacementproduct
        phaseproduct = 1
        for a, λ in enumerate((λ1, λ2)):
            phaseangle = 0
            for k in modenums:
                for j1, j2 in itertools.product(range(N),range(N)):
                    ηkj1 = η*modes[k][1][j1]
                    ηkj2 = η*modes[k][1][j2]
                    νk = modes[k][0]*νz
                    phaseangle += Ωvals[j1]*Ωvals[j2]*ηkj1*ηkj2*(t/(ωd-νk)-sin((ωd-νk)*t)/(ωd-νk)**2)*λ[j1]*λ[j2]
            phase = exp(-1j/4*phaseangle)
            if a == 1:
                phase = phase.conjugate()
            phaseproduct *= phase
        λ1λ2coeff *= phaseproduct
        λ1_index = int("".join(['0' if λ1[i]==-1 else '1' for i in range(N)]),2)
        λ2_index = int("".join(['0' if λ2[i]==-1 else '1' for i in range(N)]),2)
        ρqbit_mat[λ1_index, λ2_index] = λ1λ2coeff
        
    if fidtargets == None: fidtargets = targets
    fidtargetgatedirection = np.sign(modes[m][1][fidtargets[0]]*modes[m][1][fidtargets[1]])
    
    ρqbit = qtp.Qobj(ρqbit_mat, dims=[[2]*N,[2]*N])
    #print(ρqbit.tr())
    ρred = ρqbit.ptrace(fidtargets)
    #print(ρred.tr())
    idealUMS = (-1j*fidtargetgatedirection*π/4*tensor(sigmax(), sigmax())).expm()
    ψ0 = tensor(basis(2,0), basis(2,0))
    idealψfqbit = idealUMS*ψ0
    eigstates = σ(π/2+ϕs).eigenstates()
    change_of_basis = qtp.Qobj(np.c_[eigstates[1][0].data.A, eigstates[1][1].data.A],dims=[[2],[2]])
    idealψfqbit_zbasis = tensor(change_of_basis,change_of_basis)*idealψfqbit
    #print(ρred)
    #print(ket2dm(idealψfqbit))
    #print(ket2dm(idealψfqbit_zbasis))
    return (ρred*ket2dm(idealψfqbit_zbasis)).tr()

## Testing

We check that the analytical method of calculating fidelity matches numerical method

In [37]:
N = 5
m = 0
i1, i2 = 0, 1
targets = (i1, i2)
Ω = 1e6
idealΩvals = np.zeros(N)
idealΩvals[i1] = idealΩvals[i2] = Ω # No Crosstalk
intensities = gen_intensity_vector_fixedxt(N, (i1, i2), 0.0236**2, 0.006**2)
Ωvals = sqrt(intensities)*Ω

η = 0.01
modetype = "radial"
νz = 1e6
νr = 9e6
ϕs = π/2
ϕm = 0
modenums = [0]
modetrunc = 2
K = 1

In [38]:
# No Optical or Mode Xtalk
print(MSTracedTargetsFidelity_Analytical(N, m, targets, idealΩvals, η, modetype, νz, νr, ϕs, ϕm, [m], modetrunc, K, L=1))
print(MSTracedTargetsFidelity_FromUnitary(N, m, targets, idealΩvals, η, modetype, νz, νr, ϕs, ϕm, [m], modetrunc, K, L=1))

1.0
0.9999999999999994


In [39]:
# Optical Xtalk, No Mode Xtalk
print(MSTracedTargetsFidelity_Analytical(N, m, targets, Ωvals, η, modetype, νz, νr, ϕs, ϕm, [m], modetrunc, K, L=1))
print(MSTracedTargetsFidelity_FromUnitary(N, m, targets, Ωvals, η, modetype, νz, νr, ϕs, ϕm, [m], modetrunc, K, L=1))

0.9732939004246829
0.9732939004246828


In [40]:
# Mode Xtalk, No Optical Xtalk
print(MSTracedTargetsFidelity_Analytical(N, m, targets, idealΩvals, η, modetype, νz, νr, ϕs, ϕm, list(range(N)), modetrunc, K, L=1))
print(MSTracedTargetsFidelity_FromUnitary(N, m, targets, idealΩvals, η, modetype, νz, νr, ϕs, ϕm, list(range(N)), modetrunc, K, L=1))

0.9913212157398905
0.9913208193598034
